# Environment conda3--python3
## Coding UTF-8
### Import Libraries

In [1]:
import sys
print("Python: {}".format(sys.version))
import pandas as pd
print("pandas: {}".format(pd.__version__))
import numpy as np
print("numpy: {}".format(np.__version__))
import mysql.connector
print("connector: {}".format(mysql.connector.__version__))

Python: 3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]
pandas: 1.0.3
numpy: 1.18.1
connector: 8.0.18


### Connect to Database

In [2]:
#connect to database
con = mysql.connector.connect(
    host = "156.67.222.148",
    user = "u656477047_user",
    password = "tar15234",
    database = "u656477047_ppmb",
    port = "3306"
)

if con.is_connected():
    db_Info = con.get_server_info()
    print("Connected to MySQL Server version: ", db_Info)
    #cursor
    cur = con.cursor()
    cur.execute("select database();")
    record = cur.fetchone()
    print("You\'r connected to the database: ", record)

Connected to MySQL Server version:  5.5.5-10.2.31-MariaDB
You'r connected to the database:  ('u656477047_ppmb',)


### Execute the Query

In [3]:
SQL_Query_House = pd.read_sql_query(
    "SELECT ID_Property,PropertyType,CostestimateB,SellPrice,MarketPrice,AsseStatus,RoadType,HouseArea,Floor,HomeCondition,BuildingAge FROM propertys WHERE ID_Property='c0205'", con)
df_house = pd.DataFrame(SQL_Query_House)

if (con.is_connected()):
        #close the cursor
        cur.close()
        #close the connection
        con.close()
        print()
        print("MySQL connection is closed")
        print()


MySQL connection is closed



### Clean Data --Land_Dataset

In [4]:
# show example data from loaded file
df_house

,ID_Property,PropertyType,CostestimateB,SellPrice,MarketPrice,AsseStatus,RoadType,HouseArea,Floor,HomeCondition,BuildingAge
0,c0205,คอนโด,1300000,1430000,1700000,ว่าง,คอนกรีต,34.0,5,มือสอง,11


#### Drop Rows with missing Values

In [5]:
df_house = df_house.dropna(axis='rows')
df_house

,ID_Property,PropertyType,CostestimateB,SellPrice,MarketPrice,AsseStatus,RoadType,HouseArea,Floor,HomeCondition,BuildingAge
0,c0205,คอนโด,1300000,1430000,1700000,ว่าง,คอนกรีต,34.0,5,มือสอง,11


#### Change String to Numeric Value

In [6]:
from sklearn.preprocessing import LabelEncoder

# encoding string values into numeric values
le_propTy = LabelEncoder()
le_asseSt = LabelEncoder()
le_homeCon = LabelEncoder()
le_roadTy = LabelEncoder()

In [7]:
# create new columns containing numeric code of former column
df_house['PropertyType_n'] = le_propTy.fit_transform(df_house['PropertyType'])
df_house['AsseStatus_n'] = le_asseSt.fit_transform(df_house['AsseStatus'])
df_house['HomeCondition_n'] = le_homeCon.fit_transform(df_house['HomeCondition'])
df_house['RoadType_n'] = le_roadTy.fit_transform(df_house['RoadType'])
df_house

,ID_Property,PropertyType,CostestimateB,SellPrice,MarketPrice,AsseStatus,RoadType,HouseArea,Floor,HomeCondition,BuildingAge,PropertyType_n,AsseStatus_n,HomeCondition_n,RoadType_n
0,c0205,คอนโด,1300000,1430000,1700000,ว่าง,คอนกรีต,34.0,5,มือสอง,11,0,0,0,0


### Seperate ID_Property from Dataframe before Model Implementation

In [8]:
ID_Property = df_house[['ID_Property']]
ID_Property

,ID_Property
0,c0205


### Select Columns that are used as Variables in Model

In [9]:
df_house_select = df_house[['PropertyType_n', 'SellPrice', 'CostestimateB','MarketPrice', 'HouseArea', 'Floor', 'HomeCondition_n', 'BuildingAge','RoadType_n','AsseStatus_n']]
df_house_select

,PropertyType_n,SellPrice,CostestimateB,MarketPrice,HouseArea,Floor,HomeCondition_n,BuildingAge,RoadType_n,AsseStatus_n
0,0,1430000,1300000,1700000,34.0,5,0,11,0,0


In [10]:
df_export = df_house.to_csv(r'E:\Dataset\df_house_debug01_export.txt', index=False)

### Import Decision Tree Model

In [26]:
import pickle
print(pickle.format_version)
readdict_file = open('E:\Model\Model_Pickle_giniHouse_V03.pkl', 'rb')
classification_dict = pickle.load(readdict_file)
readdict_file.close

#loaded_model_house = pickle.load(open('E:\Model\Model_Pickle_giniHouse_V03.pkl','rb'))
#loaded_model_house = pickle.load(open("E:\Model\Model_Pickle_giniHouse_V02", 'rb'))

4.0


<function BufferedReader.close>

### Make Predictions with the Dataframe & Model

In [12]:
house_prediction = classification_dict.predict(df_house_select)
house_prediction

array([0])

In [13]:
dataframe = pd.DataFrame(house_prediction, columns=['UserType']) 
dataframe

,UserType
0,0


In [14]:
dataframe['UserType'] = np.where((dataframe.UserType==1),'Short-Term',dataframe.UserType)
dataframe

,UserType
0,0


In [15]:
dataframe = dataframe['UserType'].replace(to_replace=['0'], value='Long-Term')

In [16]:
df = pd.DataFrame(dataframe)
df

,UserType
0,Long-Term


In [18]:
#connect to database
con = mysql.connector.connect(
    host = "156.67.222.148",
    user = "u656477047_user",
    password = "tar15234",
    database = "u656477047_ppmb",
    port = "3306"
)

if con.is_connected():
    db_Info = con.get_server_info()
    print("Connected to MySQL Server version: ", db_Info)
    #cursor
    cur = con.cursor()
    cur.execute("select database();")
    record = cur.fetchone()
    print("You\'r connected to the database: ", record)

Connected to MySQL Server version:  5.5.5-10.2.31-MariaDB
You'r connected to the database:  ('u656477047_ppmb',)


In [19]:
for x,y in df_concat.iterrows():
    mycursor = con.cursor()
    cur.execute('''UPDATE propertys SET UserType =%s WHERE ID_Property=%s''', (y.UserType, y.ID_Property))
    con.commit()
    print('Index: ', x, 'Value: ', y.UserType, 'ID_P :',y.ID_Property)

Index:  0 Value:  Long-Term ID_P : c0205


In [20]:
if (con.is_connected()):
        #commit the transaction when changes made to database
        con.commit()
        #close the cursor
        cur.close()
        #close the connection
        con.close()
        print()
        print("MySQL connection is closed")
        print()


MySQL connection is closed

